# Optimizing Neural Networks with Keras Tuner on the Pima Diabetes Dataset

Neural network models are powerful tools for solving complex tasks, but finding the right architecture and hyperparameters can be a challenging and time-consuming process. Keras Tuner provides a solution to this problem by automating the hyperparameter tuning process, allowing you to efficiently search through a hyperparameter space and identify the optimal configuration for your neural network.

In this Kaggle notebook, we'll explore the application of Keras Tuner on the Pima Diabetes dataset. The dataset contains various health metrics for a group of patients, along with an indication of whether each patient developed diabetes or not. Our goal is to build and optimize a neural network to predict the onset of diabetes based on these features.

**Introduction to Keras Tuner:** Understand the basics of Keras Tuner, its key components, and how it can enhance the efficiency of your neural network development.



Load Dataset

In [30]:
import pandas as pd
import numpy as np
df = pd.read_csv('/kaggle/input/pima-diabetes/diabetes.csv')

**Exploratory Data Analysis (EDA):** Explore the Pima Diabetes dataset to gain insights into the distribution of features, identify potential challenges, and prepare the data for model training.

In [31]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [32]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [33]:
x = df.drop(['Outcome'],axis =1)
y = df['Outcome']

In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [35]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=.2,random_state=1)

# Model Building with Keras:

Construct a baseline neural network using Keras, establishing a starting point for hyperparameter tuning.



In [36]:
import tensorflow 
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [37]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [49]:
model.fit(xtrain,ytrain, batch_size=32, epochs=10,validation_split=0.2)

Epoch 1/10
16/16 [==============================] - 0s 15ms/step - loss: 0.4930 - accuracy: 0.7739 - val_loss: 0.4606 - val_accuracy: 0.7805
Epoch 2/10
16/16 [==============================] - 0s 4ms/step - loss: 0.4876 - accuracy: 0.7739 - val_loss: 0.4582 - val_accuracy: 0.7805
Epoch 3/10
16/16 [==============================] - 0s 3ms/step - loss: 0.4834 - accuracy: 0.7719 - val_loss: 0.4555 - val_accuracy: 0.7642
Epoch 4/10
16/16 [==============================] - 0s 4ms/step - loss: 0.4795 - accuracy: 0.7760 - val_loss: 0.4534 - val_accuracy: 0.7480
Epoch 5/10
16/16 [==============================] - 0s 4ms/step - loss: 0.4764 - accuracy: 0.7760 - val_loss: 0.4529 - val_accuracy: 0.7398
Epoch 6/10
16/16 [==============================] - 0s 4ms/step - loss: 0.4733 - accuracy: 0.7739 - val_loss: 0.4513 - val_accuracy: 0.7398
Epoch 7/10
16/16 [==============================] - 0s 4ms/step - loss: 0.4707 - accuracy: 0.7760 - val_loss: 0.4501 - val_accuracy: 0.7398
Epoch 8/10
16/16 [=

# Advantage Of Keras Tune
* Optimal Activation Function
 Keras Tune identifies and selects the activation function that yields the best accuracy for a given model.
* Optimal Optimizer Selection:
 Keras Tune determines the most effective optimizer for training, optimizing the model's performance based on accuracy.
* Dynamic Layer Addition:
 Keras Tune dynamically adds layers to the model based on the configuration that achieves the highest accuracy.



# Kares Tune For Select Appropriate Optimaizer

In [39]:
pip install -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [58]:
import kerastuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_model(hp):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [59]:
# making tuner object
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [62]:
tuner.search(xtrain,ytrain,epochs=5,validation_split=.02)

In [63]:
tuner.results_summary()


Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7557003498077393

Trial 2 summary
Hyperparameters:
optimizer: adam
Score: 0.7361563444137573

Trial 3 summary
Hyperparameters:
optimizer: sgd
Score: 0.7149837017059326

Trial 0 summary
Hyperparameters:
optimizer: adadelta
Score: 0.4576547145843506


# Getting Best Optimizer

In [65]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

Now you can choose the model with rms optimizer by this code

In [67]:
model = tuner.get_best_models(num_models=1)[0]

In [68]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [69]:
# Trian the model
model.fit(xtrain,ytrain, batch_size=32, epochs=100,initial_epoch=6,validation_split=0.2) # initail  epochs is noting but skip epochs 1-5 which the tunner model already run

Epoch 7/100
16/16 [==============================] - 1s 13ms/step - loss: 0.5152 - accuracy: 0.7536 - val_loss: 0.4695 - val_accuracy: 0.7967
Epoch 8/100
16/16 [==============================] - 0s 3ms/step - loss: 0.5068 - accuracy: 0.7699 - val_loss: 0.4642 - val_accuracy: 0.7967
Epoch 9/100
16/16 [==============================] - 0s 4ms/step - loss: 0.4993 - accuracy: 0.7678 - val_loss: 0.4601 - val_accuracy: 0.7805
Epoch 10/100
16/16 [==============================] - 0s 4ms/step - loss: 0.4940 - accuracy: 0.7699 - val_loss: 0.4575 - val_accuracy: 0.7805
Epoch 11/100
16/16 [==============================] - 0s 4ms/step - loss: 0.4891 - accuracy: 0.7699 - val_loss: 0.4542 - val_accuracy: 0.7886
Epoch 12/100
16/16 [==============================] - 0s 4ms/step - loss: 0.4852 - accuracy: 0.7719 - val_loss: 0.4523 - val_accuracy: 0.7886
Epoch 13/100
16/16 [==============================] - 0s 4ms/step - loss: 0.4817 - accuracy: 0.7780 - val_loss: 0.4502 - val_accuracy: 0.7967
Epoch 14

# Building Tune Model For Get Best Neuron Numbers

In [106]:
def build_model1(hp):
    model = Sequential()
    units = hp.Int('units', min_value=8, max_value=128, step=8)
    model.add(Dense(units=units, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# create tuner object
tuner1 = kt.RandomSearch(build_model1,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name='tuner')

tuner1.search(xtrain, ytrain, epochs=5, validation_split=0.2)


Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.6016260385513306

Best val_accuracy So Far: 0.7886179089546204
Total elapsed time: 00h 00m 08s


In [107]:
tuner1.results_summary()

Results summary
Results in mydir/tuner
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
units: 80
Score: 0.7886179089546204

Trial 2 summary
Hyperparameters:
units: 112
Score: 0.772357702255249

Trial 0 summary
Hyperparameters:
units: 120
Score: 0.7560975551605225

Trial 1 summary
Hyperparameters:
units: 56
Score: 0.7479674816131592

Trial 4 summary
Hyperparameters:
units: 8
Score: 0.6016260385513306


In [108]:
tuner1.get_best_hyperparameters()[0].values

{'units': 80}

In [109]:
#Getting best model
model = tuner1.get_best_models(num_models=1)[0]

In [110]:
model.fit(xtrain,ytrain,batch_size=32,epochs=100,initial_epoch=5,validation_split=.02)

Epoch 6/100
19/19 [==============================] - 1s 10ms/step - loss: 0.5144 - accuracy: 0.7621 - val_loss: 0.6676 - val_accuracy: 0.6154
Epoch 7/100
19/19 [==============================] - 0s 3ms/step - loss: 0.4962 - accuracy: 0.7671 - val_loss: 0.6638 - val_accuracy: 0.6154
Epoch 8/100
19/19 [==============================] - 0s 4ms/step - loss: 0.4833 - accuracy: 0.7720 - val_loss: 0.6596 - val_accuracy: 0.6154
Epoch 9/100
19/19 [==============================] - 0s 3ms/step - loss: 0.4744 - accuracy: 0.7770 - val_loss: 0.6587 - val_accuracy: 0.6154
Epoch 10/100
19/19 [==============================] - 0s 3ms/step - loss: 0.4676 - accuracy: 0.7787 - val_loss: 0.6577 - val_accuracy: 0.6154
Epoch 11/100
19/19 [==============================] - 0s 3ms/step - loss: 0.4625 - accuracy: 0.7787 - val_loss: 0.6612 - val_accuracy: 0.6154
Epoch 12/100
19/19 [==============================] - 0s 4ms/step - loss: 0.4591 - accuracy: 0.7804 - val_loss: 0.6617 - val_accuracy: 0.6154
Epoch 13/

# Tune Model For Select Layers

In [112]:
def build_model2(hp):
    model = Sequential()
    model.add(Dense(80,activation='relu',input_dim=8))
    for i in range(hp.Int('num_layers',min_value=1,max_value=8)):
        model.add(Dense(80,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [113]:
tuner2 = kt.RandomSearch(build_model2,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='myd',
                        project_name='layers')

In [114]:
tuner2.search(xtrain,ytrain,epochs=5,validation_split=0.2)

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7560975551605225

Best val_accuracy So Far: 0.7804877758026123
Total elapsed time: 00h 00m 11s


In [116]:
tuner2.results_summary()

Results summary
Results in myd/layers
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
num_layers: 8
Score: 0.7804877758026123

Trial 2 summary
Hyperparameters:
num_layers: 1
Score: 0.772357702255249

Trial 3 summary
Hyperparameters:
num_layers: 4
Score: 0.772357702255249

Trial 0 summary
Hyperparameters:
num_layers: 7
Score: 0.7642276287078857

Trial 4 summary
Hyperparameters:
num_layers: 5
Score: 0.7560975551605225


In [117]:
tuner2.get_best_hyperparameters(1)[0].values

{'num_layers': 8}

In [118]:
#Training Model From Tuner2 Function
model = tuner2.get_best_models(num_models=1)[0]

In [119]:
model.fit(xtrain,ytrain,epochs=100,validation_split=0.2,initial_epoch=5)

Epoch 6/100
16/16 [==============================] - 1s 18ms/step - loss: 0.4578 - accuracy: 0.7841 - val_loss: 0.4852 - val_accuracy: 0.7724
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4430 - accuracy: 0.8045 - val_loss: 0.4675 - val_accuracy: 0.7724
Epoch 8/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4313 - accuracy: 0.7760 - val_loss: 0.4851 - val_accuracy: 0.7805
Epoch 9/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4287 - accuracy: 0.8106 - val_loss: 0.4607 - val_accuracy: 0.7642
Epoch 10/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4175 - accuracy: 0.8065 - val_loss: 0.4721 - val_accuracy: 0.7480
Epoch 11/100
16/16 [==============================] - 0s 6ms/step - loss: 0.3981 - accuracy: 0.8147 - val_loss: 0.4700 - val_accuracy: 0.7642
Epoch 12/100
16/16 [==============================] - 0s 5ms/step - loss: 0.3770 - accuracy: 0.8350 - val_loss: 0.5250 - val_accuracy: 0.7398
Epoch 13/

**You Can See Accuracy Got 1 and Val_accuracy reduce to 64 From 76**

# Build Tune Function To Get All Perfect Perameter

In [129]:
from keras.layers import Dense, Dropout
def build_model3(hp):
    model = Sequential()
    counter = 0
    
    # Loop through the specified number of layers
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        if counter == 0:
            # For the first layer, include input_dim and dropout
            model.add(
                Dense(
                    hp.Int('Units' + str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']),
                    input_dim=8
                )
            )
            model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])))
        else:
            # For subsequent layers, include dropout
            model.add(
                Dense(
                    hp.Int('Units' + str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])
                )
            )
            model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])))
        counter += 1

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model with tunable hyperparameters
    model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [130]:
tuner3 = kt.RandomSearch(build_model3,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mtd',
                        project_name='final')

Reloading Tuner from mtd/final/tuner0.json


In [131]:
tuner3.search(xtrain,ytrain,epochs=5,validation_split=0.2)

In [132]:
tuner3.results_summary()

Results summary
Results in mtd/final
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 4 summary
Hyperparameters:
num_layers: 8
Units0: 16
activation0: sigmoid
optimizer: adam
Units1: 96
activation1: tanh
Units2: 40
activation2: relu
Units3: 16
activation3: relu
Units4: 80
activation4: tanh
Units5: 16
activation5: tanh
Units6: 64
activation6: sigmoid
Units7: 104
activation7: sigmoid
Units8: 64
activation8: sigmoid
Units9: 128
activation9: sigmoid
Score: 0.7804877758026123

Trial 1 summary
Hyperparameters:
num_layers: 7
Units0: 8
activation0: sigmoid
optimizer: adam
Units1: 8
activation1: relu
Units2: 8
activation2: relu
Units3: 8
activation3: relu
Units4: 8
activation4: relu
Units5: 8
activation5: relu
Units6: 8
activation6: relu
Score: 0.6341463327407837

Trial 3 summary
Hyperparameters:
num_layers: 10
Units0: 48
activation0: relu
optimizer: adadelta
Units1: 56
activation1: sigmoid
Units2: 72
activation2: sigmoid
Units3: 96
activation3: sigmoid
Units4: 8
ac

In [133]:
tuner3.get_best_hyperparameters(1)[0].values

{'num_layers': 8,
 'Units0': 16,
 'activation0': 'sigmoid',
 'optimizer': 'adam',
 'Units1': 96,
 'activation1': 'tanh',
 'Units2': 40,
 'activation2': 'relu',
 'Units3': 16,
 'activation3': 'relu',
 'Units4': 80,
 'activation4': 'tanh',
 'Units5': 16,
 'activation5': 'tanh',
 'Units6': 64,
 'activation6': 'sigmoid',
 'Units7': 104,
 'activation7': 'sigmoid',
 'Units8': 64,
 'activation8': 'sigmoid',
 'Units9': 128,
 'activation9': 'sigmoid'}

In [134]:
final_model = tuner3.get_best_models(num_models=1)[0]

In [135]:
final_model.fit(xtrain,ytrain,epochs=100,validation_split=0.2,initial_epoch=5)

Epoch 6/100
16/16 [==============================] - 2s 18ms/step - loss: 0.5608 - accuracy: 0.7088 - val_loss: 0.5269 - val_accuracy: 0.7724
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5675 - accuracy: 0.7067 - val_loss: 0.4967 - val_accuracy: 0.7561
Epoch 8/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5431 - accuracy: 0.6904 - val_loss: 0.5027 - val_accuracy: 0.7561
Epoch 9/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5508 - accuracy: 0.7088 - val_loss: 0.5067 - val_accuracy: 0.7886
Epoch 10/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5294 - accuracy: 0.7373 - val_loss: 0.4851 - val_accuracy: 0.7480
Epoch 11/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5512 - accuracy: 0.7088 - val_loss: 0.4988 - val_accuracy: 0.7561
Epoch 12/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5576 - accuracy: 0.7373 - val_loss: 0.4940 - val_accuracy: 0.7480
Epoch 13/

# Why Keras Tuner?
Keras Tuner offers a user-friendly and flexible platform for hyperparameter tuning, enabling data scientists and machine learning practitioners to unlock the full potential of their neural network models. By the end of this notebook, you'll have a solid understanding of how to leverage Keras Tuner to enhance the performance of your neural networks and accelerate your model development workflow.

Let's dive in and discover the art of optimizing neural networks with Keras Tuner!
